<a href="https://colab.research.google.com/github/palash04/Artificial-Intelligence/blob/master/NN_Series/_Pytorch_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [2]:
# set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
# Hyperparameters
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

In [4]:
# Create RNN
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN,self).__init__()

    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

    # Forward pass
    ###print ('x shape: ', x.shape)
    out, _ = self.gru(x, h0)
    ###print ('out shape: ', out.shape)
    out = out.reshape(x.shape[0], -1)
    ###print ('out shape reshaped: ', out.shape)
    out = self.fc(out)

    return out


In [5]:
# Load data
train_dataset = datasets.MNIST(root='dataset/', train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

test_dataset = datasets.MNIST(root='dataset/', train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [6]:
# Initialize the RNN
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
print (model)




RNN(
  (gru): GRU(28, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=7168, out_features=10, bias=True)
)


In [7]:
# Testing the model
x = torch.randn(64,1,28,28)
#print (x.shape)
out = model(x.to(device).squeeze(1))
#print (out.shape)

In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [9]:
# Train the Network
for epoch in range(1,num_epochs+1):
  training_loss = 0.0
  for batch_idx, (data,targets) in enumerate(train_loader):
    # get data to cuda if possible
    data = data.to(device=device).squeeze(1)
    targets = targets.to(device=device)

    # forward pass
    output = model(data)

    # calculate loss
    loss = criterion(output, targets)

    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    training_loss += loss.item()
  
  print (f'Epoch {epoch}/{num_epochs} \t Trainging Loss: {training_loss/len(train_loader):.6f}')


Epoch 1/2 	 Trainging Loss: 0.195739
Epoch 2/2 	 Trainging Loss: 0.064552


In [10]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print ('Checking for training data')
  else:
    print ('Checking for testing data')

  num_correct = 0
  num_samples = 0
  model.eval()
  with torch.no_grad():
    for x,y in loader:
      x = x.to(device).squeeze(1)
      y = y.to(device)

      output = model(x)
      _, preds = output.max(1)
      num_correct += (preds == y).sum()
      num_samples += preds.size(0)

    print (f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples) * 100 :.6f}')
  model.train()
  

In [11]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking for training data
Got 59186/60000 with accuracy 98.643333
Checking for testing data
Got 9852/10000 with accuracy 98.520000


In [12]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(LSTM, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True)
    self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
  
  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0) ,self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0) ,self.hidden_size).to(device)
    
    # Forward pass
    out, _ = self.lstm(x, (h0, c0))
    out = out.reshape(x.shape[0], -1)
    out = self.fc(out)
    return out

In [13]:
model = LSTM(input_size, hidden_size,num_layers, num_classes).to(device)
print (model)

LSTM(
  (lstm): LSTM(28, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=7168, out_features=10, bias=True)
)


In [14]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [15]:
!pip install tqdm

In [16]:
from tqdm import tqdm

In [17]:
# Train the Network
num_epochs = 2
for epoch in range(1,num_epochs+1):
  training_loss = 0.0
  loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
  for batch_idx, (data,targets) in loop:
    # get data to cuda if possible
    data = data.to(device=device).squeeze(1)
    targets = targets.to(device=device)

    # forward pass
    output = model(data)

    # calculate loss
    loss = criterion(output, targets)

    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    training_loss += loss.item()
  
    # update progress bar
    loop.set_description(f'Epoch [{epoch}/{num_epochs}]')
    loop.set_postfix(loss = loss.item())    # can also include acc
  # print (f'Epoch {epoch}/{num_epochs} \t Trainging Loss: {training_loss/len(train_loader):.6f}')


In [19]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking for training data
Got 59165/60000 with accuracy 98.608333
Checking for testing data
Got 9854/10000 with accuracy 98.540000
